<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-\amily:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Data Leakage
              
</p>
</div>

Data Science Cohort Live NYC June 2022
<p>Phase 3: Topic 24</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error,

#### Data Leakage
- When information not generally available/used at prediction time contaminates the modeling training process.

<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> The despair of realizing you have a data leak.</center>

Leads to overconfident estimates of model performance during the validation and testing phases.

- Bad performance after model deployment in production.


<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> My overlords are upset.</center>

<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> I have spent all night trying to find the data leak.</center>

<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> What is the nature of my leak?</center>

<center><img src = "Images/despair.jpg" width = 350 /></center>
<center> Where is it??</center>

Diagnosing a data leak can be subtle business:
- Understanding different types of leak
- Where in the process they can be accidentally introduced.

#### Training leakage

Fitting and applying transformations **before** train-test(hold out) splitting.

<img src = "Images/training_testing.png" width = 500 />

- Why is this bad?

Applying transformation to training set:
- contains information from test set (contained in fitted attributes of Transformer)

- Contaminated training set.
- Model has inadvertently trained on information you are trying to predict.

#### You can introduce it by accident on really innocuous steps.

In [ ]:
titanic_df = pd.read_csv('Data/titanic.csv').drop(columns = ['Cabin'])
titanic_df.info()

Let's impute those NaNs in age with the mean.

In [ ]:
# impute with the mean
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean())
titanic_df.info()

Bam. You just introduced data leakage.
- imputed with statistics of entire dataset before train-test split.
- Statistics of train in test and vice-versa.
- Whether impact is significant will depend on data and model.

#### Leakage between train and true test can be a truly costly mistake:
- You'll only figure it out after you've deployed
- Made suboptimal predictions:
    - incorrectly recommending inventory changes to increase sales
    - or *much* worse.


#### Leakage in cross validation

Less bad but still not good...

<img src = "Images/crossval.png" width = 500/>

When you pollute your training fold with your validation fold:

- Each cross-validation trial has data leakage.


Validation performance measurements not correct:
- Incorrect estimates of average model performance
- Incorrect estimates of model variance.

Messes up your hyperparameter tuning:
- "Best model": hyperparameter settings with best average model performance
- But it doesn't work well on my test/hold-out set...

#### Then check for data leakage in your cross-validation

#### Other types of data leakage

#### Feature leakage

Dependent on how the data/features were collected.

<img src = "Images\data_leakage_predcontam.png" width = 800/>

- Predicting whether we should approve a loan or not.
- Have database of current lendees.

- Features include:
    - Salary information at time of loan approval
    - Pre-loan bankruptcy history
    - FICO score
    - Listed occupation
    - Monthly interest payments.
    - Bank balance across accounts.
    

**What's the problem here?**

Possibly conflating information from *after* prediction with properties before prediction.
- Features from **after** prediction potentially affected by target.
- Our target is now leaking into the way we trained our model.
- Features *before* and *after* approval may not be drawn from same distribution.

**These sources of feature leakage can be very subtle**
- Understanding of the data collection process and data definitions is critical here.

**Case Study**

Predict sale price of a given home.

- Size of the house (in square meters)
- Average sales price of homes in the same neighborhood
- Latitude and longitude of the house
- Whether the house has a basement

Is there a source of potential data leakage? Explain.

Depends on whether the average includes the sales prices including given sale or before it.

**Another Case Study**

Want to predict rate of infection during surgery based on:
- patient specific factors (immuno-history, etc.)
- environmental factors (hospital cleanliness inspections, etc.)

Your idea: include surgeon as a factor.

1. Take all surgeries by each surgeon and calculate the infection rate among those surgeons.
2. For each patient in the data, find out who the surgeon was and plug in that surgeon's average infection rate as a feature.

Potentially introduces both target leakage and train-test leak:

- Target leakage: if given patient's outcome contributes to the infection rate for his surgeon. 
- Using target to calculate feature.
- Then using this feature to predict target.

Can avoid by using:
- Surgeon's infection rate for only surgeries before the patient we are predicting for.
- Tricky, for sure.

Train-test contamination problem if you calculate this using all surgeries a surgeon performed
- Including those from the test-set. 

Are you tearing your hair out? Good. You now understand data leakage.

## Preprocessing

In general all preprocessing steps are subject to the same dangers here. Consider the preprocessing step of one-hot-encoding:

In [ ]:
gun_poll = pd.read_csv('data/guns-polls.csv')

In [ ]:
gun_poll.head()

In [ ]:
gun_poll['Pollster'].value_counts()

Now if I were to fit a one-hot encoder to the whole `Pollster` column here, the encoder would learn all the categories. But I need to prepare myself for the real-world possibility that unfamiliar categories may show up in future records. Let's explore this.

In [ ]:
# First I'll do a split
X_train, X_test = train_test_split(gun_poll, random_state=42)

Let's suppose now that I fit a `OneHotEncoder` to the `Pollster` column in my training data.

#### Exercise
Fit an encoder to the `Pollster` column of the training data and then check to see which categories are represented.

<details>
    <summary>Answer</summary>
<code>to_be_dummied = X_train[['Pollster']]
ohe = OneHotEncoder()
ohe.fit(to_be_dummied)
## So what categories do we have?
ohe.get_feature_names()</code>
</details>

We'll want to transform both train and test after we've fitted the encoder to the train.

In [ ]:
test_to_be_dummied = X_test[['Pollster']]

ohe.transform(to_be_dummied)
ohe.transform(test_to_be_dummied)

There are categories in the testing data that don't appear in the training data! What should 
we do about that?

### Approaches
- **Strategy 1**: Divide up the categories proportionally when we do our train_test_split. If we're using `sklearn`'s tool, that means taking advantage of the `stratify` parameter:

In [ ]:
new_X_train, new_X_test = train_test_split(gun_poll,
                                           stratify=gun_poll['Pollster'],
                                           random_state=42)

Unfortunately, in this case, we can't use this since some categories have only a single member.

- **Strategy 2**: Drop the categories with very few representatives.

In the present case, let's try dropping the single-member categories.

In [ ]:
vc = gun_poll['Pollster'].value_counts()
vc_only_1 = vc[vc == 1]
bad_cols = vc_only_1.index

In [ ]:
bad_cols

In [ ]:
gun_poll['Pollster'] = gun_poll['Pollster'].map(lambda x: np.nan if x in bad_cols else x)
gun_poll = gun_poll.dropna()

In [ ]:
gun_poll['Pollster'].value_counts()

We could now split this carefully so that new categories don't show up in the testing data. In fact, now we can try the stratified split:

In [ ]:
X_train3, X_test3 = train_test_split(gun_poll,
                                     stratify=gun_poll['Pollster'],
                                     test_size=0.3,
                                     random_state=42)

In [ ]:
X_train3['Pollster'].value_counts()

In [ ]:
X_test3['Pollster'].value_counts()

Now every category that appears in the test data appears also in the training data.

- **Strategy 3**: Adjust the settings on the one-hot-encoder.

For `sklearn`'s tool, we'll tweak the `handle_unknown` parameter:

#### Exericse
Fit a new encoder to our training data column that won't break when we try to use it to transform the test data. And then use the encoder to transform both train and test.

<details>
    <summary>Answer</summary>
<code>ohe2 = OneHotEncoder(handle_unknown='ignore')
ohe2.fit(to_be_dummied)
test_to_be_dummied = X_test[['Pollster']]
ohe2.transform(to_be_dummied)
ohe2.transform(test_to_be_dummied)</code>
</details>

In [ ]:
t = pd.DataFrame(ohe2.transform(test_to_be_dummied).todense(), columns = ohe2.get_feature_names())
t.head()

In [ ]:
data = load_diabetes()

print(data.DESCR)

df = pd.concat([pd.DataFrame(data.data, columns=data.feature_names),
               pd.Series(data.target, name='target')], axis=1)

df.head()

In [ ]:
X, y = load_diabetes(return_X_y=True)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=42)

In [ ]:
for train_ind, val_ind in KFold().split(X_train2):
    
    train = X_train2[train_ind, :]
    val = X_train2[val_ind, :]
    
    target_train = y_train2[train_ind]
    target_val = y_train2[val_ind]
    
    ss = StandardScaler().fit(train)
    
    train_scld = ss.transform(train)
    
    val_scld = ss.transform(val)
    
    lr = LinearRegression().fit(train_scld, target_train)
    
    print(lr.coef_[0])

### A Contrived But Illustrative Example

In [ ]:
fake_preds = np.array([6, 5, 2, 250, 300]).reshape(-1, 1)
fake_target = np.array([25, 30, 12, 400, 420]).reshape(-1, 1)

In [ ]:
df = pd.DataFrame(np.hstack((fake_preds, fake_target)),
                 columns=['pred', 'target'])

In [ ]:
df

In [ ]:
small_train = df[['pred']][:3]
small_val = df[['pred']][3:]
small_train_y = df['target'][:3]
small_val_y = df['target'][3:]

In [ ]:
# Scaling the whole dataset

ss = StandardScaler().fit(df[['pred']])

In [ ]:
X_tr = ss.transform(small_train)
X_va = ss.transform(small_val)

In [ ]:
lr = LinearRegression().fit(X_tr, small_train_y)
lr.coef_

In [ ]:
fig, ax = plt.subplots()

X = np.linspace(0, 310, 600)
y = lr.coef_ * X + lr.intercept_

ax.scatter(df['pred'], df['target'])
ax.plot(X, y)
ax.set_title(f"""The best-fit line when scaling the whole dataset is
    {round(lr.coef_[0])}X + {round(lr.intercept_)}""");

In [ ]:
# Splitting before scaling

ss2 = StandardScaler().fit(small_train)

In [ ]:
X_tr2 = ss2.transform(small_train)
X_va2 = ss2.transform(small_val)

In [ ]:
lr2 = LinearRegression().fit(X_tr2, small_train_y)
lr2.coef_

In [ ]:
fig, ax = plt.subplots()

X = np.linspace(0, 310, 600)
y = lr2.coef_ * X + lr2.intercept_

ax.scatter(df['pred'], df['target'])
ax.plot(X, y)
ax.set_title(f"""The best-fit line when scaling after splitting is
    {round(lr2.coef_[0])}X + {round(lr2.intercept_)}""");